### Create perturbed and unperturbed trajectories

In [1]:
from os import listdir
from os.path import join, isdir
import pickle
import yaml
import numpy as np
np.set_printoptions(linewidth=100)

In [2]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot
from interpolators import InterpolatorFactory

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt

In [4]:
# # Adiabatic SSM settings
# ROMOrder = 3
# # N_samples = 100
# DT = 0.01
# INTERPOLATE = "reduced_coords" # "xyz" # "xy" # "reduced_coords"

In [5]:
observables = "delay-embedding" # "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = join(robot_dir, 'rest_qv.pkl')

# # load rest position
# with open(rest_file, 'rb') as f:
#     rest_data = pickle.load(f)
#     rest_q = rest_data['q'][0]

In [6]:
ALPHA = 3.0

Track an ideal circle using a previous controller (SSMR).

In [7]:
traj_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/autonomous_ASSM_tests"
(t, z), u = utils.import_pos_data(data_dir=join(traj_dir, "circle_ideal_using-ssmr-controller", f"alpha={ALPHA:.1f}"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True)
print("z.shape:", z.shape)
print("u.shape:", u.shape)

z.shape: (3, 511)
u.shape: (8, 511)


Smoothen inputs

In [8]:
from copy import deepcopy

In [9]:
def smooth(weights, arr):
   # pad with left and right values to avoid boundary effects
   arr_padded = np.concatenate((arr[0]*np.ones(len(weights)//2), arr, arr[-1]*np.ones(len(weights)//2-1)))
   return np.convolve(weights/weights.sum(), arr_padded, mode='valid')

In [10]:
u_smooth = deepcopy(u)
smooth_window_size = 20
for i in range(u.shape[0]):
    u_smooth[i, :] = smooth(np.hanning(smooth_window_size), u[i, :]) # [smooth_window_size//2:1-smooth_window_size//2]

Increase sampling rate

In [11]:
# from scipy.interpolate import interp1d
# td = np.linspace(t[0], t[-1], len(t)*10)
# f = interp1d(t, u_smooth, kind='linear')
# ud_smooth = f(td)

Plot the ideal trajectory

In [12]:
plt.close('all')
plot.inputs(t, u)
plot.inputs(t, u_smooth)
# plot.inputs(td, ud_smooth)

Simulate two trajectories from this: one using the smoothened inputs ($z_0$); one using the smoothened inputs but with a perturbation before starting the trajectory ($z_0+\varepsilon$)

In [13]:
with open(join(traj_dir, "OL_sim_ideal", f"alpha={ALPHA:.1f}", "u_ideal.pkl"), "wb") as f:
    model = pickle.dump(u_smooth, f)
# with open(join(traj_dir, "u_ideal_1ms.pkl"), "wb") as f:
#     model = pickle.dump(ud_smooth, f)

In [14]:
# add perturbation to idle 10ms inputs
perturbation = np.array([50, 0, 0, 0, 50, 0, 0, 0]).reshape(-1, 1)

# 10 ms
u_perturbed = deepcopy(u_smooth)
u_perturbed[:, 100:300] += perturbation
with open(join(traj_dir, "OL_sim_perturbed", f"alpha={ALPHA:.1f}", "u_perturbed.pkl"), "wb") as f:
    model = pickle.dump(u_perturbed, f)
    
# # 1ms
# ud_perturbed = deepcopy(ud_smooth)
# ud_perturbed[:, 1000:2000] += perturbation
# with open(join(traj_dir, "u_perturbed_1ms.pkl"), "wb") as f:
#     model = pickle.dump(ud_perturbed, f)

In [15]:
plt.close('all')
plot.inputs(t, u_perturbed)
plot.inputs(t, u_smooth)
# plot.inputs(td, ud_perturbed)

Finally, do open-loop simulations using these two input sequences in SOFA using:
- u_perturbed
- u_ideal

Save as OL_sim_ideal_snapshots in the correct folders corresponding to the given alpha value.